## 1.处理数据

In [ ]:
mkdir -p /root/sft/Retrieval-based-Voice-Conversion-WebUI/logs/leijun 

In [ ]:
!python infer/modules/train/preprocess.py \
"/root/sft/MSST-WebUI/results/leijun_25min/vocals_noreverb/" \
40000 \
15 \
"/root/sft/Retrieval-based-Voice-Conversion-WebUI/logs/leijun" \
False \
3.7

## 2.提取特征

### 2.1 提取f0

In [ ]:
!python \
    "infer/modules/train/extract/extract_f0_rmvpe.py" \
    1 \
    0 \
    0 \
    "/root/sft/Retrieval-based-Voice-Conversion-WebUI/logs/leijun" \
    True

### 2.2 提取feature

In [ ]:
!python \
    "infer/modules/train/extract_feature_print.py" \
    "cuda:0" \
    1 \
    0 \
    0 \
    "/root/sft/Retrieval-based-Voice-Conversion-WebUI/logs/leijun" \
    "v2" \
    True

## 3.训练

In [ ]:
!python generate_filelist.py -e logs/leijun25min


In [ ]:
!python \
    "infer/modules/train/train.py" \
    -e "leijun25min" \
    -sr 40k \
    -f0 1 \
    -bs 32 \
    -g 0 \
    -te 300 \
    -se 100 \
    -pg assets/pretrained_v2/f0G40k.pth \
    -pd assets/pretrained_v2/f0D40k.pth \
    -l 0 \
    -c 0 \
    -sw 0 \
    -v v2  \



### 3.1 训练索引

In [ ]:
!python \
train_index.py\
-e leijun25min \
-v v2 \
-o /root/sft/Retrieval-based-Voice-Conversion-WebUI/assets/indices

## 推理

In [ ]:
!python tools/infer_cli.py \
  --input_path "/root/sft/DDSP-SVC/input/31948473856-1-192_vocals_noreverb.wav" \
  --opt_path "output.wav" \
  --model_name "leijun25min.pth" \
  --f0up_key -10 \
  --f0method "rmvpe" \
  --index_path "/root/sft/Retrieval-based-Voice-Conversion-WebUI/assets/indices/leijun25min_IVF1673_Flat_nprobe_1_leijun25min_v2.index" \
  --index_rate 0.75 \
  --device "cuda:0" \
  --is_half True\
  --rms_mix_rate 0.25